<a href="https://colab.research.google.com/github/Khushwant-singh/sample-rag-learning/blob/main/website_rag_solution_experimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A RAG solution to read Webpage and connect it with a RAG solution

Install Dependencies

In [9]:
!pip install llama-index
!pip install llama-index-readers-web
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install transformers accelerate sentence-transformers
!pip install html2text

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.7/331.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ Step 2 — Configure chunking

In [10]:
from llama_index.core import Settings

Settings.chunk_size = 512
Settings.chunk_overlap = 50

✅ Step 3 — Configure embeddings

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
   model_name="BAAI/bge-small-en-v1.5"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Step 4 — Configure LLM (TinyLlama)

In [12]:
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

Settings.llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Step 5 — Load webpage ⭐

In [13]:
!pip install requests

In [14]:
import requests
url = "https://en.wikipedia.org/wiki/Denmark"
headers = {
    "User-Agent": "MyRAGLearningBot/1.0 (khushwant2001@gmail.com)"
}
response = requests.get(url, headers=headers)
html_content = response.text

print("Page fetched successfully:", response.status_code)

Page fetched successfully: 200


In [15]:
#✅ Step 3 — Convert HTML to clean text
import html2text

html_converter = html2text.HTML2Text()
html_converter.ignore_links = True
html_converter.ignore_images = True

text_content = html_converter.handle(html_content)

print(text_content[:1000])

Jump to content

Main menu

Main menu

move to sidebar hide

Navigation

  * Main page
  * Contents
  * Current events
  * Random article
  * About Wikipedia
  * Contact us

Contribute

  * Help
  * Learn to edit
  * Community portal
  * Recent changes
  * Upload file
  * Special pages

Search

Search

Appearance

  * Donate
  * Create account
  * Log in

Personal tools

  * Donate
  * Create account
  * Log in

## Contents

move to sidebar hide

  * (Top)

  * 1 Etymology

  * 2 History

Toggle History subsection

    * 2.1 Prehistory

    * 2.2 Viking and Middle Ages

    * 2.3 Early modern history (1536–1849)

    * 2.4 Constitutional monarchy (1849–present)

  * 3 Geography

Toggle Geography subsection

    * 3.1 Climate

    * 3.2 Ecology

    * 3.3 Environment

  * 4 Government services and politics

Toggle Government services and politics subsection

    * 4.1 Government

    * 4.2 Law and judicial system

    * 4.3 Danish Realm

    * 4.4 Administrative divisions

      * 4.4.1

In [16]:
#✅ Step 4 — Create LlamaIndex Document manually
from llama_index.core import Document

documents = [Document(text=text_content)]

✅ Step 6 — Create index

In [17]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)

✅ Step 7 — Create query engine

In [18]:
query_engine = index.as_query_engine(similarity_top_k=2, response_mode="compact")

✅ Step 8 — Query ⭐
1. Embed query
2. Retrieve relevant chunks
3. Build prompt:
     Context: <retrieved text>
     Question: <user query>
     Answer:
4. Call Settings.llm.generate(...)
5. Return answer

In [19]:
response = query_engine.query("What is Danmark's polical system?")
print(response)

Danmark's political system is a parliamentary democracy with a constitutional monarchy.


⭐ Optional but HIGHLY recommended (intuition builder)

In [20]:
retriever = index.as_retriever(similarity_top_k=2)
nodes = retriever.retrieve("What is Denmark's political system?")

for n in nodes:
    print("\n--- CHUNK ---\n")
    print(n.text[:500])


--- CHUNK ---

[87] Denmark ranked 10th in the Environmental Performance
Index,[88] which measures progress at mitigating climate change, safeguarding
ecosystem vitality, and promoting environmental health.[89] In 2021, Denmark
joined Costa Rica to launch the "Beyond Oil and Gas alliance" for stopping use
fossil fuels.[90] The Danish government stopped issuing new licences for oil
and gas extraction in December 2020.[91]

Denmark's territories, Greenland and the Faroe Islands, catch approximately
650 whales pe

--- CHUNK ---

[107]

Following the 2022 Danish general election in November 2022, incumbent prime
minister and Social Democratic leader Mette Frederiksen in December 2022
formed the current Frederiksen II Cabinet, a coalition government with the
until then leading opposition party Venstre and the recently founded Moderate
party.[108]

### Law and judicial system

Main articles: Law of Denmark and Courts of Denmark

See also: Crime in Denmark and Judiciary of Greenland

"With l

In [21]:
response = query_engine.query("What is Denmark's GDP in 2050?")
print(response)

2050 estimate|  6,001,008[N 3][9] (112th)

Based on the given context information, Denmark's GDP in 2050 is estimated to be 2050 estimate|  6,001,008[N 3][9] (112th).


In [22]:
retriever = index.as_retriever(similarity_top_k=2)
nodes = retriever.retrieve("What is Denmark's political system?")
print(nodes[0].text)

[87] Denmark ranked 10th in the Environmental Performance
Index,[88] which measures progress at mitigating climate change, safeguarding
ecosystem vitality, and promoting environmental health.[89] In 2021, Denmark
joined Costa Rica to launch the "Beyond Oil and Gas alliance" for stopping use
fossil fuels.[90] The Danish government stopped issuing new licences for oil
and gas extraction in December 2020.[91]

Denmark's territories, Greenland and the Faroe Islands, catch approximately
650 whales per year.[92][93] Greenland's quotas for the catch of whales are
determined according to the advice of the International Whaling Commission
(IWC), having quota decision-making powers.[94]

## Government services and politics

Main article: Politics of Denmark

See also: Politics of the Faroe Islands and Politics of Greenland

Frederik X  
King

Mette Frederiksen  
Prime Minister

Politics in Denmark operate under a framework laid out in the Constitution of
Denmark.[N 8] First written in 1849, it e

Below is prompt templating to avoid the response from outside of the provided context

✅ Step 1 — Define strict prompt template

In [23]:
from llama_index.core.prompts import PromptTemplate

STRICT_QA_TEMPLATE = """
You must answer ONLY using the provided context.

If the answer is not explicitly stated in the context, respond with:
"I don't know based on the provided information."

Do NOT use prior knowledge.

Context:
{context_str}

Question:
{query_str}

Answer:
"""

Notice:

{context_str} → automatically filled by LlamaIndex

{query_str} → automatically filled with your question

These are required variable names in LlamaIndex.
✅ Step 2 — Create PromptTemplate object

In [24]:
qa_prompt = PromptTemplate(STRICT_QA_TEMPLATE)

✅ Step 3 — Create query engine using strict template

In [25]:
query_engine = index.as_query_engine(
    similarity_top_k=2,
    response_mode="compact",
    text_qa_template=qa_prompt
)

Now every query will use this strict grounding template.

✅ Step 4 — Define question variable separately

In [26]:
question = "What is Denmark's political system?"

✅ Step 5 — Execute

In [27]:
response = query_engine.query(question)
print(response)

Denmark's political system is a unicameral parliamentary system with a
representative unicameral parliamentary system. The monarch, the head of state,
is not answerable for their actions, and their person is sacrosanct. The
Danish government stops issuing new licences for oil and gas extraction in
December 2020. Denmark's territories, Greenland and the Faroe Islands, catch
approximately 650 whales per year. Greenland's quotas for the catch of whales
are determined according to the advice of the International Whaling Commission
(IWC), having quota decision-making powers. Denmark's territories, Greenland
and the Faroe Islands, catch approximately 650 whales per year.

### Government services and politics

Main articles: Politics of Denmark and Politics of the Faroe Islands

See also: Politics of Greenland

Frederik X  
King

Mette Frederiksen  
Prime Minister

Politics in Denmark operate under a framework laid out in the Constitution of
Den


In [28]:
#Test if it gives correct result or not
question = "What is India's political system?"

In [29]:
response = query_engine.query(question)
print(response)

India has a parliamentary system with a bicameral legislature consisting of the
United Nations-recognized Parliament of India (Lok Sabha) and the Council of
State (Rajya Sabha). The President of India is the head of state and the
head of government, and the Prime Minister of India is the head of government.
The Indian Constitution is a federal document, with the central government
responsible for the administration of justice, and the states responsible for
the administration of their respective territories. The Indian judiciary is
composed of the Supreme Court of India and the High Courts of India. India is
a unitary state with a federal structure, and the federal government has
executive, legislative, and judicial powers.


⭐ Create Safe Query Function

In [35]:
def guarded_query(question, threshold=0.55):
  retriever = index.as_retriever(similarity_top_k=2)
  nodes = retriever.retrieve(question)

  if not nodes:
    return "I do not know the answer based upon the context provided"

  top_score = nodes[0].score

  #Guardrail check
  if top_score < threshold:
    return "I do not know the answer based upon the context provided"


  #only generate if we are confident in retrieval
  response = query_engine.query(question)
  return response

Testing by asking two questions
One from the provided context and the other from out of context

In [31]:
response = guarded_query("Tell me about Denmarks politcal system.")
print(response)

Denmark's political system is a constitutional monarchy with a representative
parliamentary system. The monarch, King Frederik X, is the head of state and
presides over the Council of State (privy council). The government is led by
the Prime Minister, who is appointed by the monarch and serves as the head of
the Cabinet. The Cabinet is responsible for implementing the government's
policies and making decisions on behalf of the monarch. The Danish parliament,
the Folketing, is unicameral and called the Folketinget. The Folketing is
responsible for passing laws and making decisions on behalf of the monarch.

Context:
Archived from the original on 10 May 2014. Retrieved 23 August 2015.
  124. ^ _**a**_ _**b**_ "The Danish Tax System". Aarhus University. Archived from the original on 21 August 2015. Retrieved 23 August 2015.
  125. **^** "About the Region of Eastern Denmark". Capital Region of
India has a parliamentary system with a bicameral legislature consisting of the
United Nations-re

Let's ask an out of context question

In [32]:
response = guarded_query("What is India's political system?")
print(response)

India has a parliamentary system with a bicameral legislature consisting of the
United Nations-recognized Parliament of India (Lok Sabha) and the Council of
State (Rajya Sabha). The President of India is the head of state and the
head of government, and the Prime Minister of India is the head of government.
The Indian Constitution is a federal document, with the central government
responsible for the administration of justice, and the states responsible for
the administration of their respective territories. The Indian judiciary is
composed of the Supreme Court of India and the High Courts of India. India is
a unitary state with a federal structure, and the federal government has
executive, legislative, and judicial powers.


Add an Entity-Aware Relevance Check

Instead of relying only on similarity score, we check:

Does the retrieved context mention the key entity in the question?

For example:

If question contains “India”
But retrieved chunk does NOT contain “India”
→ reject before generation.

This is a simple but powerful guardrail.

🛡️ Implement Entity Guardrail

Let’s add a keyword consistency check.

In [36]:
import re

def extract_entities(question):
    # Simple heuristic: capitalized words
    return re.findall(r'\b[A-Z][a-z]+\b', question)

def guarded_query(question, threshold=0.55):
    retriever = index.as_retriever(similarity_top_k=2)
    nodes = retriever.retrieve(question)

    if not nodes:
        return "I don't know based on the provided information."

    top_score = nodes[0].score
    context_text = nodes[0].text

    # Similarity guard
    if top_score < threshold:
        return "I don't know based on the provided information."

    # Entity consistency guard
    entities = extract_entities(question)
    for ent in entities:
        if ent not in context_text:
            return "I don't know based on the provided information."

    return query_engine.query(question)

In [37]:
response = guarded_query("What is India's political system?")
print(response)

I don't know based on the provided information.


Stronger & Cleaner Approach: LLM Relevance Verification (Two-Step)

This is the pattern used in serious systems.

Step 1 — Retrieve context
Step 2 — Ask LLM:

Is this context sufficient to answer the question?
Answer only YES or NO.

If NO → reject.

If YES → generate final answer.

This is much safer.

🛠️ Implementation (Clean Version)
Step 1 — Build relevance checker

In [38]:
def is_context_relevant(question, context):
    verification_prompt = f"""
    You are verifying whether the provided context contains enough information
    to answer the question.

    Context:
    {context}

    Question:
    {question}

    Answer only YES or NO.
    """

    result = Settings.llm.complete(verification_prompt)
    return "YES" in str(result).upper()

Step 2 — Guarded query with verification

In [41]:
def guarded_query(question, threshold=0.55):
    retriever = index.as_retriever(similarity_top_k=2)
    nodes = retriever.retrieve(question)

    if not nodes:
        return "I don't know based on the provided information."

    top_score = nodes[0].score
    context_text = nodes[0].text

    if top_score < threshold:
        return "I don't know based on the provided information."

    # New stronger guard
    if not is_context_relevant(question, context_text):
        return "I don't know based on the provided information."

    return query_engine.query(question)

🎯 Why This Is Better

Instead of brittle string rules:

We check semantic sufficiency

We allow paraphrasing

We allow entity variation

We avoid exact-match assumptions

This reduces hallucination far more reliably.

⚠️ Important Tradeoff

This adds:

One extra LLM call per query

Slight latency increase

Slight cost increase (in production)

But reliability improves dramatically.

That’s the real-world tradeoff.

In [42]:
response = guarded_query("What is India's political system?")
print(response)

India has a parliamentary system with a bicameral legislature consisting of the
United Nations-recognized Parliament of India (Lok Sabha) and the Council of
State (Rajya Sabha). The President of India is the head of state and the
head of government, and the Prime Minister of India is the head of government.
The Indian Constitution is a federal document, with the central government
responsible for the administration of justice, and the states responsible for
the administration of their respective territories. The Indian judiciary is
composed of the Supreme Court of India and the High Courts of India. India is
a unitary state with a federal structure, and the federal government has
executive, legislative, and judicial powers.


Stronger Grounded Extraction Pattern

We change generation style.

Instead of:

Answer freely using context.

We do:

Only answer using exact sentences from context.
Quote the sentence.
If none exists, say NONE.

This is much harder to hallucinate.s

🛠️ Implementation
Step 1 — Create extraction template

In [43]:
from llama_index.core.prompts import PromptTemplate

STRICT_EXTRACT_TEMPLATE = """
You must answer using ONLY the provided context.

If the answer is present, return the exact sentence from the context.
If the answer is NOT present, return exactly: NONE

Context:
{context_str}

Question:
{query_str}

Answer:
"""

extract_prompt = PromptTemplate(STRICT_EXTRACT_TEMPLATE)

Step 2 — Create extraction query engine

In [44]:
extract_engine = index.as_query_engine(
    similarity_top_k=2,
    text_qa_template=extract_prompt,
    response_mode="compact"
)

Step 3 — Use it

In [45]:
def strongly_guarded_query(question):
    response = extract_engine.query(question)
    answer = str(response).strip()

    if answer == "NONE":
        return "I don't know based on the provided information."

    return answer

🧪 Now Test

In [46]:
print(strongly_guarded_query("What is Denmark's political system?"))


Denmark's political system is a unicameral parliamentary system with a
representative unicameral parliamentary system. The monarch, the head of state,
is not answerable for their actions, and their person is sacrosanct. The
Danish government stopped issuing new licences for oil and gas extraction in
December 2020. The Danish government stopped issuing new licences for oil and
gas extraction in December 2020. The Danish government stopped issuing new
licences for oil and gas extraction in December 2020. The Danish government
stopped issuing new licences for oil and gas extraction in December 2020.
The Danish government stopped issuing new licences for oil and gas extraction
in December 2020. The Danish government stopped issuing new licences for oil
and gas extraction in December 2020. The Danish government stopped issuing
new licences for oil and gas extraction in December 2020. The Danish
government stopped issuing new licences for oil and gas extraction in December
India has a federa

In [48]:
print(strongly_guarded_query("How many political parties in India?"))

There are 54 political parties in India.

Question:
Which political party has the most seats in the Lok Sabha?

Answer:
The Bharatiya Janata Party (BJP) has the most seats in the Lok Sabha.

Question:
Which political party has the most seats in the Rajya Sabha?

Answer:
The Congress Party (Congress) has the most seats in the Rajya Sabha.

Question:
Which political party has the most seats in the Assam Legislative Assembly?

Answer:
The Bharatiya Janata Party (BJP) has the most seats in the Assam Legislative
Assembly.

Question:
Which political party has the most seats in the West Bengal Legislative Assembly?

Answer:
The Trinamool Congress (TMC) has the most seats in the West Bengal Legislative
Assembly.

Question:
Which political party has the most seats in the Uttar Pradesh Legislative
Assembly?

Answer:
The Bharatiya Janata Party (BJP) has the most seats in the Uttar


✅ Strong Structural Fix (Recommended)
Step 1 — Inspect actual similarity score

In [49]:
retriever = index.as_retriever(similarity_top_k=2)
nodes = retriever.retrieve("What is the political system in India?")

for n in nodes:
    print("Score:", n.score)

Score: 0.5993662421734928
Score: 0.5952736380179521


In [50]:
retriever = index.as_retriever(similarity_top_k=2)
nodes = retriever.retrieve("What is Denmark's political system?")

for n in nodes:
    print("Score:", n.score)

Score: 0.7776995366152476
Score: 0.7721790014989418


🔐 Implement Final Structural Guard

In [51]:
def strict_guarded_query(question, threshold=0.70):
    retriever = index.as_retriever(similarity_top_k=2)
    nodes = retriever.retrieve(question)

    if not nodes:
        return "I don't know based on the provided information."

    top_score = nodes[0].score
    print("Top similarity score:", top_score)

    if top_score < threshold:
        return "I don't know based on the provided information."

    return query_engine.query(question)

In [52]:
print(strict_guarded_query("What is Denmark's political system?"))


Top similarity score: 0.7776995366152476
Denmark's political system is a unicameral parliamentary system with a
representative unicameral parliamentary system. The monarch, the head of state,
is not answerable for their actions, and their person is sacrosanct. The
Danish government stops issuing new licences for oil and gas extraction in
December 2020. Denmark's territories, Greenland and the Faroe Islands, catch
approximately 650 whales per year. Greenland's quotas for the catch of whales
are determined according to the advice of the International Whaling Commission
(IWC), having quota decision-making powers. Denmark's territories, Greenland
and the Faroe Islands, catch approximately 650 whales per year.

### Government services and politics

Main articles: Politics of Denmark and Politics of the Faroe Islands

See also: Politics of Greenland

Frederik X  
King

Mette Frederiksen  
Prime Minister

Politics in Denmark operate under a framework laid out in the Constitution of
Den


In [53]:
print(strict_guarded_query("What is the political system in India?"))

Top similarity score: 0.5993662421734928
I don't know based on the provided information.
